# GIFs aus der DDB erstellen

Die [Deutsche Digitale Bibliothek](https://www.deutsche-digitale-bibliothek.de/) stellt große Mengen an Bildmaterial zur kreativen Nachnutzung zur Verfügung.

Im folgenden Beispiel wird demonstriert, wie aus einer Treffermenge ein GIF erzeugt werden kann.
Als Bilder werden die im Rahmen des [digiS-Projekts "Die Sichtbarmachung des Sichtbaren – Berlins typografisches Kulturerbe im Open Access"](https://www.digis-berlin.de/projektpartner/archiv/projektpartner-2021/#tab-tb_qu2x406-9) von Erik Spiekermann erstellten Schriftproben verwendet.

In [ ]:
# Import der Module
from lxml import etree
from glob import glob
import requests
from ddbAPI_helpers import search2API, iterateAPICall # ddbAPI_helpers.py muss aktuell noch im selben Ordner wie das Notebook/Skript liegen

## Download der Bilder

In einem ersten Schritt sollen die Bilder zu einer bestimmten Suchanfrage heruntergeladen werden.
Benötigt wird hierfür die URL (`searchurl`) der Suchanfrage und der [API-Key der DDB](https://www.deutsche-digitale-bibliothek.de/login?referrer=%2Fuser%2Fapikey) (`api_key`).

In [21]:
api_key = "<API-KEY>"

In [ ]:
searchurl = "https://www.deutsche-digitale-bibliothek.de/searchresults?query=dataset_id%3A10904480765851755bPLD&offset=0&_=1659951819810&rows=20&facetValues%5B%5D=objecttype_fct%3DSchriftprobe&facetValues%5B%5D=affiliate_fct_role%3Dp98a"

Mit der Funktion `search2API` wird aus der URL ein API-Call generiert und dieser dann mit `iterateAPICall` iterativ durchgeführt, bis alle Objekte geladen wurden.
Die Variable `targetdir` bezeichnet den Zielordner, in dem die zu den Objekten gehörigen LIDO-XML-Dateien gespeichert werden.

In [ ]:
apiurl = search2API(searchurl, api_key)
objects = iterateAPICall(apiurl, api_key, download = True, targetdir = "schriftproben")

## Extraktion der Bild-URLs aus den LIDO-Dateien

Damit die Bilder in größtmöglicher Auflösung heruntergeladen werden können, muss die jeweilige Bild-URL aus den LIDO-Dateien extrahiert werden.
Im Fall unseres Datensatzes befindet sich die entsprechende Adresse im XML unter:

```
<lido:resourceWrap>
      <lido:resourceSet lido:sortorder="1">
        <lido:resourceRepresentation lido:type="image_master">
          <lido:linkResource lido:formatResource="image/jpeg">https://berlin.museum-digital.de/data/berli/images/17/202203/109742/v7-0028-109742.jpg</lido:linkResource>
        </lido:resourceRepresentation>
      </lido:resourceSet>
</lido:resourceWrap>
          
```

In [ ]:
# Definition des LIDO-Namespace
NSMAP = { 'lido' : "http://www.lido-schema.org"}

Der folgende Abschnitt liest die heruntergeladenen XML-Dateien ein, extrahiert die jeweilige Bild-URL (in die Variable `img_url`), lädt das Bild herunter (`requests.get(img_url)`) und speichert sie im selben Ordner ab.

In [ ]:
files = glob('schriftproben/*.xml')
for o in files:
    tree = etree.parse(o)
    img_url = tree.find('//lido:resourceRepresentation[@lido:type="image_master"]/lido:linkResource', NSMAP).text
    target_path = o.replace('.xml','.jpg')
    with open(target_path, 'wb') as OUT:
        res = requests.get(img_url)
        OUT.write(res.content)

Die Bilder liegen nun im Ordner `schriftproben`.

## Erzeugung der GIFs

GIFs lassen sich mit dem [Pillow-Modul](https://pillow.readthedocs.io) erzeugen.

Wir iterieren also über die heruntergeladenen Bilddateien und lesen sie als `Image`-Objekt ein.
Um die GIF-Datei nicht allzu groß werden zu lassen, können die Einzelbilder mit der `thumbnail`-Methode auf ein bestimmtes Maximalmaß reduziert werden.



<div class="alert alert-block alert-info">Der Code hierfür wurde von <a href="https://stackoverflow.com/questions/60562248/making-a-gif-using-pil-pillow">hier</a> adaptiert.</div>

In [ ]:
from PIL import Image

In [ ]:
frames = []
imgs = glob("schriftproben/*.jpg")
for i in imgs:
    new_frame = Image.open(i)
    new_frame.thumbnail((200,300))
    frames.append(new_frame)
    
frames[0].save(
    'img/SCHRIFTPROBE.gif',
    format='GIF',
    append_images=frames[0:],
    save_all=True,
    duration=200,
    loop=0,
    optimize=False,
    transparency=0
)

## Das Ergebnis

![Ergebnis](img/SCHRIFTPROBE.gif)